# Python `with` Statement and Azure AI Agents

🎯 **Understanding context management with Azure AI Agents!**

This tutorial will teach you about the Python `with` statement and how it applies to Azure AI Agents:

1. **What is the `with` statement?** - Understanding context managers
2. **Foundry SDK Examples** - Using `with` with `azure.ai.agents`
3. **Semantic Kernel Examples** - Using `with` with `semantic_kernel.agents`
4. **When to use `with` vs when NOT to use it** - Best practices

**Perfect for developers who want to write cleaner, more reliable code!** 🚀

---

## 🧠 What is the Python `with` Statement?

The `with` statement is Python's way of handling **context management**. Think of it as an automatic "setup and cleanup" mechanism.

### 🔄 What happens with `with`:
1. **Enter**: Resources are set up (connections opened, credentials loaded)
2. **Work**: Your code runs inside the `with` block
3. **Exit**: Resources are automatically cleaned up (connections closed, credentials disposed)

### 💡 Key Benefits:
- **Automatic cleanup**: No need to remember to close connections
- **Exception safety**: Cleanup happens even if errors occur
- **Cleaner code**: Less boilerplate, more readable

### 🎯 Perfect for:
- Database connections
- File operations
- API clients (like Azure AI Agents!)
- Any resource that needs cleanup

Let's see this in action! 🚀

## 📋 Setup and Prerequisites

Before we start, make sure you have:

### Environment Variables:
- `PROJECT_ENDPOINT`: Your Azure AI Project endpoint
- `MODEL_DEPLOYMENT_NAME`: Your deployed AI model name

### Required Packages:
We'll install both the Foundry SDK and Semantic Kernel packages.

In [ ]:
# If you haven't created the environment and installed the requirements, install the required packages
# This might take a minute or two

# !pip install azure-ai-agents azure-identity

# Check if the required packages are installed
import importlib.metadata
for package in ["semantic-kernel", "azure-identity", "azure-ai-agents"]:
    try:
        version = importlib.metadata.version(package)
        print(f"✅ {package} is installed (version {version}).")
    except importlib.metadata.PackageNotFoundError:
        print(f"❌ {package} is NOT installed.")
        
print("Now let's explore the 'with' statement with Azure AI Agents!")

In [ ]:
# Import everything we need
import os
import asyncio
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import AgentThreadCreationOptions, ThreadMessageOptions
from azure.identity import DefaultAzureCredential
from azure.identity.aio import DefaultAzureCredential as AsyncDefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings

print("📦 All packages imported!")

# Set environment variables if needed (uncomment and fill in):
# os.environ['PROJECT_ENDPOINT'] = 'https://your-project.cognitiveservices.azure.com/'
# os.environ['MODEL_DEPLOYMENT_NAME'] = 'your-model-deployment-name'

required_vars = ['AZURE_OPENAI_ENDPOINT', 'AZURE_OPENAI_API_KEY', 'AZURE_OPENAI_DEPLOYMENT_NAME', 'PROJECT_ENDPOINT', 'MODEL_DEPLOYMENT_NAME']
missing_vars = []

for var in required_vars:
    if var not in os.environ:
        missing_vars.append(var)
    else:
        print(f"✅ {var} is set")

if missing_vars:
    print(f"\n❌ Missing environment variables: {missing_vars}")
    print("\n🔧 Please set them using:")
    for var in missing_vars:
        print(f"   os.environ['{var}'] = 'your_value_here'")
else:
    print("\n🎉 All environment variables are properly configured!")

---

## 🔧 Example 1: Foundry SDK - Using `with` Statement

Let's start with the **recommended approach** using the `with` statement with the Foundry SDK.

### ✅ **Why use `with` here?**
- **Automatic credential cleanup**: Azure credentials are properly disposed
- **Connection management**: HTTP connections are closed automatically
- **Exception safety**: Resources cleaned up even if something goes wrong
- **Best practice**: Microsoft recommends this pattern

In [ ]:
# ✅ RECOMMENDED: Using 'with' statement with Foundry SDK

print("🔧 Example 1: Foundry SDK WITH the 'with' statement")
print("═" * 60)

# Creating the agents client 
agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential()
)

# The 'with' statement automatically handles setup and cleanup
with agents_client:
    
    print("🚀 Inside the 'with' block - client is active")
    
    # Create an agent
    agent = agents_client.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="with-statement-demo",
        instructions="You are a helpful assistant that explains Python concepts clearly. Your responses should be very consise and clear."
    )
    
    print(f"🤖 Agent created: {agent.name}")
    
    # Have a quick conversation
    run = agents_client.create_thread_and_process_run(
        agent_id=agent.id,
        thread=AgentThreadCreationOptions(
            messages=[ThreadMessageOptions(
                role="user", 
                content="Explain what a context manager is in one sentence."
            )]
        )
    )
    
    if run.status == "completed":
        messages = agents_client.messages.list(thread_id=run.thread_id)
        for msg in messages:
            if msg.role == "assistant" and msg.text_messages:
                print(f"🤖 Agent response: {msg.text_messages[-1].text.value}")
                break
    
    # Cleanup agent
    agents_client.delete_agent(agent.id)
    print(f"🧹 Agent deleted")

print("✅ Exited 'with' block - client automatically cleaned up!")
print("💡 Credentials disposed, connections closed automatically")
print("\n" + "═" * 60)

## 🚫 Example 2: Foundry SDK - WITHOUT `with` Statement

Now let's see what happens when we **don't** use the `with` statement.

### ⚠️ **Why you might avoid `with`:**
- **Long-lived applications**: When you need the client to persist across multiple functions
- **Shared clients**: When multiple parts of your app use the same client
- **Custom lifecycle management**: When you want full control over when resources are cleaned up

### ❗ **Important**: You're responsible for cleanup!

In [ ]:
# ⚠️ WITHOUT 'with' statement - Manual resource management

print("🔧 Example 2: Foundry SDK WITHOUT the 'with' statement")
print("═" * 60)

# Create client manually (no automatic cleanup)
agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential()
)

print("🚀 Client created manually - we're responsible for cleanup")

try:
    # Create an agent
    agent = agents_client.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="manual-management-demo",
        instructions="You are a helpful assistant. Your responses should be very consise and clear."
    )
    
    print(f"🤖 Agent created: {agent.name}")
    
    # Quick interaction
    run = agents_client.create_thread_and_process_run(
        agent_id=agent.id,
        thread=AgentThreadCreationOptions(
            messages=[ThreadMessageOptions(
                role="user", 
                content="What's the benefit of automatic resource management?"
            )]
        )
    )
    
    if run.status == "completed":
        messages = agents_client.messages.list(thread_id=run.thread_id)
        for msg in messages:
            if msg.role == "assistant" and msg.text_messages:
                print(f"🤖 Agent response: {msg.text_messages[-1].text.value}")
                break
    
    # Manual cleanup
    agents_client.delete_agent(agent.id)
    print(f"🧹 Agent deleted manually")
    
except Exception as e:
    print(f"❌ Error occurred: {e}")
    
finally:
    # IMPORTANT: Manual cleanup of the client
    # In real code, you'd call client.close() or similar
    print("🔧 We should manually close the client here")
    print("💡 This is why 'with' is often better - it's automatic!")

print("\n" + "═" * 60) 

---

## 🔄 Example 3: Semantic Kernel - Async `with` Statement

The Semantic Kernel approach uses **async/await** patterns, which also support the `with` statement through **async context managers**.

### 🌟 **Key differences with Semantic Kernel:**
- **Async pattern**: Uses `async with` instead of just `with`
- **Multiple context managers**: Credentials AND client both use `with`
- **More explicit**: Separate credential and client management

Let's see this in action:

In [ ]:
# ✅ RECOMMENDED: Semantic Kernel with async 'with' statements

async def semantic_kernel_with_example():
    print("🔧 Example 3: Semantic Kernel WITH async 'with' statements")
    print("═" * 60)
    
    # Notice the nested 'async with' statements!
    async with (
        AsyncDefaultAzureCredential() as creds,  # Credential context manager
        AzureAIAgent.create_client(credential=creds, endpoint=os.environ["PROJECT_ENDPOINT"]) as client,  # Client context manager
    ):
        print("🚀 Inside nested 'async with' blocks - both credential and client active")
        
        # Create agent definition
        agent_definition = await client.agents.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"],
            name="semantic-kernel-demo",
            instructions="You are a helpful assistant that understands async programming. Your responses should be very consise and clear.",
        )
        
        print(f"🤖 Agent definition created: {agent_definition.name}")
        
        # Create Semantic Kernel agent wrapper
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Have conversation
        response = await agent.get_response(
            messages="What makes async programming powerful?"
        )
        
        print(f"🤖 Agent response: {response}")
        
        # Cleanup
        if response.thread:
            await response.thread.delete()
        await client.agents.delete_agent(agent.id)
        print(f"🧹 Agent and thread cleaned up")
    
    print("✅ Exited 'async with' blocks - everything automatically cleaned up!")
    print("💡 Both credentials AND client properly disposed")
    print("\n" + "═" * 60)

# Run the async example
await semantic_kernel_with_example()

---

## 🔄 Example 4: Semantic Kernel - Without the `with` Statement

⚠️ WITHOUT 'async with' statements - Manual resource management with Semantic Kernel

In [ ]:
print("🔧 Example 4: Semantic Kernel WITHOUT 'async with' statements")
print("═" * 60)

async def semantic_kernel_without_example():
    # Create credential and client manually (no automatic cleanup)
    credential = AsyncDefaultAzureCredential()
    client = AzureAIAgent.create_client(credential=credential, endpoint=os.environ["PROJECT_ENDPOINT"])
    
    print("🚀 Client created manually - we're responsible for cleanup")
    
    try:
        # Create agent definition
        agent_definition = await client.agents.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"],
            name="sk-manual-management-demo",
            instructions="You are a helpful assistant that explains resource management. Your responses should be very consise and clear.",
        )
        
        print(f"🤖 Agent definition created: {agent_definition.name}")
        
        # Create Semantic Kernel agent wrapper
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Have conversation
        response = await agent.get_response(
            messages="What are the risks of not cleaning up resources properly?"
        )
        
        print(f"🤖 Agent response: {response}")
        
        # Manual cleanup
        if response.thread:
            await response.thread.delete()
        await client.agents.delete_agent(agent.id)
        print(f"🧹 Agent and thread cleaned up manually")
        
    except Exception as e:
        print(f"❌ Error occurred: {e}")
        
    finally:
        # IMPORTANT: Manual cleanup of the client and credential
        await client.close()
        await credential.close()
        print("🔧 Manually closed client and credential")
        print("💡 This is why 'async with' is often better - it handles this automatically!")

print("Starting manual Semantic Kernel example...")
await semantic_kernel_without_example()
print("✅ Example completed!")
print("\n" + "═" * 60)

---

## 🎯 When to Use `with` vs When NOT to Use It

### ✅ **USE `with` when:**

1. **Short-lived operations**: Single request-response interactions
2. **Simple scripts**: One-off tasks or utilities
3. **Clear boundaries**: When you know exactly when you're done with the client
4. **Exception safety is critical**: When you want guaranteed cleanup
5. **Following best practices**: Microsoft recommends this pattern

```python
# Perfect for 'with'
with AgentsClient(...) as client:
    agent = client.create_agent(...)
    result = client.create_thread_and_process_run(...)
    client.delete_agent(agent.id)
# Automatic cleanup happens here
```

### ⚠️ **AVOID `with` when:**

1. **Long-lived applications**: Web servers, background services
2. **Shared clients**: Multiple functions/classes need the same client
3. **Custom lifecycle**: You need precise control over when cleanup happens
4. **Performance critical**: Creating/destroying clients frequently is expensive
5. **Complex state management**: When client lifetime doesn't match your scope

```python
# Better without 'with' for long-lived apps
class ChatService:
    def __init__(self):
        self.client = AgentsClient(...)  # Lives for app lifetime
        self.agent = self.client.create_agent(...)
    
    def chat(self, message):
        return self.client.create_thread_and_process_run(...)
    
    def shutdown(self):
        self.client.close()  # Manual cleanup when app shuts down
```

## 🏆 Best Practices Summary

### 🎯 **Golden Rules:**

1. **Default to `with`**: Use it unless you have a specific reason not to
2. **Match your pattern**: 
   - Script/utility → Use `with`
   - Long-lived app → Manual management
3. **Handle exceptions**: Always cleanup, even when errors occur
4. **Async awareness**: Use `async with` for async code

### 🔍 **Quick Decision Guide:**

**Ask yourself**: *"Does my client lifetime match this code block?"*
- **Yes** → Use `with` ✅
- **No** → Manual management ⚠️

### 🚀 **Performance Tips:**

- **Reuse clients** when possible (expensive to create)
- **Use connection pooling** for high-throughput scenarios  
- **Monitor resource usage** to catch leaks early

### 💡 **Remember:**

The `with` statement isn't just about cleanup—it's about **writing more reliable, maintainable code**. When in doubt, use `with`! 🎉

---

## 🎓 Congratulations!

You now understand how to use the Python `with` statement effectively with Azure AI Agents!

### ✅ **What You've Learned:**

- **Context managers**: What they are and why they matter
- **Foundry SDK patterns**: Both `with` and manual approaches
- **Semantic Kernel patterns**: Async context management
- **Decision making**: When to use each approach
- **Best practices**: Writing reliable, maintainable code

### 🚀 **Next Steps:**

- Apply these patterns in your own projects
- Experiment with long-lived vs short-lived client scenarios
- Explore advanced context management patterns
- Check out Azure AI Agents function calling and streaming

### 💡 **Key Takeaway:**

**Use `with` by default** for cleaner, safer code. Only manage resources manually when you have a specific architectural need.

**Happy coding with Azure AI Agents!** 🎉✨